In [1]:
from ultralytics import YOLO
# from picamera import PiCamera
from time import sleep
import numpy as np
import cv2

In [ ]:
camera = PiCamera()
name = "base"
i = 0

In [ ]:
camera.capture("C:\Users\zhakl\Desktop\Thesis\Thesis\baseline" + str(i) + ".jpg")

In [2]:
capture = cv2.VideoCapture("utube.mp4")
success, frame = capture.read()

In [3]:

# Load a pretrained YOLOv8n model
model = YOLO("C:/Users/zhakl/Desktop/Thesis/Thesis/weights/road.pt")

# Define path to the image file
source = 'pedestrian-crossing-webtile.jpg'

n = 0

# Run inference on the source
results = model(source, save=True)  # list of Results objects


image 1/1 c:\Users\zhakl\Desktop\Thesis\Thesis\Pedestrian-Crossing.jpg: 448x640 1 SA, 137.0ms
Speed: 1.0ms preprocess, 137.0ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\segment\predict252


In [5]:
base = YOLO("yolov8n-seg.pt")

source = 'test1.jpg'

n = 0

# Run inference on the source
results = base(source, save=True) 


image 1/1 c:\Users\zhakl\Desktop\Thesis\Thesis\test1.jpg: 480x640 7 cars, 1 truck, 143.3ms
Speed: 3.1ms preprocess, 143.3ms inference, 13.8ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\segment\predict242


In [6]:
# Extract bounding boxes, classes, object names, confidence scores, and masks
boxes = results[0].boxes.xyxy.tolist()
classes = results[0].boxes.cls.tolist()
names = results[0].names
confidences = results[0].boxes.conf.tolist()
masks = results[0].masks

h, w = results[0].orig_shape
# points = np.empty()
# Iterate through each detected object's box, class, confidence, and mask


ltc, ltdc, pc, 
11, 2, 4

In [7]:
print(names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

2 3 5 7 

In [5]:
import matplotlib.pyplot as plt

img = cv2.imread(source)


H, W, _ = img.shape


In [5]:
points_list = []

minX = []
minY = []
maxX = []
maxY = []

In [6]:
for i, tup in enumerate(zip(boxes, classes, confidences, masks)):
    _, cls, _, mask = tup

    minX.append(9999)
    minY.append(9999)
    maxX.append(0)
    maxY.append(0)
    for point in mask.xy[0]:
        if minX[i] > point[0]:
            minX[i] = point[0]
        if minY[i] > point[1]:
            minY[i] = point[1]
        if maxX[i] < point[0]:
            maxX[i] = point[0]
        if maxY[i] < point[1]:
            maxY[i] = point[1]
    points_list.append(mask.xy[0])

In [7]:
print(minX)
print(minY)
print(maxX)
print(maxY)

[632.0, 1224.0, 192.0]
[144.0, 386.0, 472.0]
[1278.0, 1278.0, 446.0]
[228.0, 720.0, 720.0]


In [8]:
def detect_collision(x, y):
    # print("asda")
    global points_list
    dist = -1
    for i, points in enumerate(points_list):
        # print("asd")
        if x > maxX[i] or x < minX[i]:
            continue
        if y > maxY[i] or y < minY[i]:
            continue        
        dist = cv2.pointPolygonTest(points, (int(x),int(y)), False)
        if dist > 0:
            return dist
    return dist

In [27]:
print(len(points_list))

3


In [ ]:
camera.start_preview()
camera.start_recording('/home/pi/Desktop/video.h264')
sleep(5)
camera.stop_recording()
camera.stop_preview()

In [8]:
detect_collision(150,298)

asda
asd
asd
asd


-1.0

In [9]:
def predict(frame):
    result = model(frame, save=True, classes=[7])
    return result




In [10]:
plate = YOLO("C:/Users/zhakl/Desktop/Thesis/Thesis/weights/plate.pt")
count = 0


In [11]:
def plate_detection(img):
    global count
    results = plate(img, save=True) 
    boxes = results[0].boxes.xyxy.tolist()
    for box in boxes:
        object = [int(box[0]), int(box[1]), int(box[2]- box[0]), int(box[3] - box[1])]
        x,y,w,h = object   
        cv2.imwrite('detected_plate_' + str(count) + '.jpg', img[y:y+h, x:x+w])
        count += 1

In [12]:
while success:
    success,image = capture.read()
    n += 1
    if n >= 10:
        n = 0
        result = predict(image)
        res_classes = result[0].boxes.cls.tolist()
        res_masks = result[0].masks
        res_boxes = result[0].boxes.xyxy.tolist()
        for i,tup in enumerate(zip(res_boxes,res_classes,res_masks)):
            box, cls, mask = tup
            points = (mask.xy[0])
            image_1 = np.zeros((h, w, 3), dtype=np.uint8)
            for point in points:
                # print (point[0], point[1])
                if detect_collision(point[0], point[1]) > 0:
                    object = [int(box[0]), int(box[1]), int(box[2]- box[0]), int(box[3] - box[1])]
                    x,y,w,h = object
                    plate_detection(image[y:y+h, x:x+w])
                    break


0: 384x640 2 VEHICLEs, 127.0ms
Speed: 8.0ms preprocess, 127.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\segment\predict48

0: 608x640 (no detections), 132.0ms
Speed: 3.0ms preprocess, 132.0ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)
Results saved to runs\detect\predict25

0: 384x640 2 VEHICLEs, 103.0ms
Speed: 1.0ms preprocess, 103.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\segment\predict48

0: 608x640 (no detections), 150.0ms
Speed: 4.0ms preprocess, 150.0ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 640)
Results saved to runs\detect\predict25

0: 384x640 2 VEHICLEs, 91.0ms
Speed: 2.0ms preprocess, 91.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\segment\predict48

0: 576x640 (no detections), 139.0ms
Speed: 4.0ms preprocess, 139.0ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)
Results s

In [29]:
print(points_list)

[array([[        868,         144],
       [        866,         146],
       [        854,         146],
       [        852,         148],
       [        848,         148],
       [        846,         150],
       [        842,         150],
       [        840,         148],
       [        838,         148],
       [        836,         146],
       [        820,         146],
       [        818,         148],
       [        816,         148],
       [        814,         150],
       [        796,         150],
       [        794,         148],
       [        792,         148],
       [        790,         146],
       [        766,         146],
       [        764,         148],
       [        762,         148],
       [        760,         150],
       [        754,         150],
       [        752,         152],
       [        744,         152],
       [        742,         150],
       [        732,         150],
       [        730,         152],
       [        634